In [1]:
from edge_analyzer.acquisition import airt
from edge_analyzer import channel_waveform

duration = 0.5
sample_rate_Hz = 15.36e6
analysis_bandwidth_Hz = 10e6

print('connecting...')
capture = airt.AirTCapture(frequency_Hz=2.44e9, sample_rate_Hz=sample_rate_Hz)

try:
    print('acquiring...')
    raw_iq = capture.acquire(int(duration*sample_rate_Hz))
finally:
    print('done')
    capture.close()

connecting...
acquiring...
done


In [2]:
# import importlib
# from edge_analyzer import channel_waveform
# channel_waveform = importlib.reload(channel_waveform)

analysis_spec = {
    'power_time_series': {'detector_period': 10e-3, 'detectors': ('rms', 'peak')},
    'cyclic_channel_power': {
        'cyclic_period': 10e-3,
        'detector_period': 1e-3 / 15 / 4,
        'detectors': ('rms', 'peak'),
        'cyclic_statistics': ('min', 'mean', 'max'),
    },
    'persistence_spectrum': {
        'window': 'flattop',
        'resolution': 15e3,
        'quantiles': [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 0.999, 1],
        'truncate': True,
    },
    'amplitude_probability_distribution': {
        'power_low': -40,
        'power_high': 15,
        'power_count': 221,  # 0.25 dB resolution
    },    
}

filter_params = dict(
    passband_ripple_dB=0.1,
    stopband_attenuation_dB=90,
    transition_bandwidth_Hz=250e3,
)

channel_waveform.from_spec(
    raw_iq,
    sample_rate_Hz,
    analysis_bandwidth_Hz=analysis_bandwidth_Hz,
    filter_spec=filter_params,
    analysis_spec=analysis_spec,
)

ValueError: conflicting sizes for dimension 'baseband_frequency': length 667 on the data but length 1024 on coordinate 'baseband_frequency'

In [ ]:
%%timeit -n1 -r1

channel_waveform.from_spec(
    raw_iq,
    sample_rate_Hz,
    analysis_bandwidth_Hz=analysis_bandwidth_Hz,
    filter_spec=filter_spec,
    analysis_spec=analysis_spec,
)